# Textual Analysis

In [1]:
%load_ext autoreload
%autoreload 2

import plotly.io as pio
pio.renderers.keys()
pio.renderers.default = 'notebook' 

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.express as px
from itertools import chain
import re
import nltk
nltk.download('punkt')

from utils.eda import *
from utils.text_model import transform_text, tokenize

import plotly.graph_objects as go
from plotly.subplots import make_subplots

[nltk_data] Downloading package punkt to /Users/kevinb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# text data
fall_2022_text = pd.read_csv('data/2022_fall_text.csv')
winter_2023_text = pd.read_csv('data/2023_winter_text.csv')
spring_2023_text = pd.read_csv('data/2023_spring_text.csv')
summer_2023_text = pd.read_csv('data/2023_summer_text.csv')
fall_2023_text = pd.read_csv('data/2023_fall_text.csv')
winter_2024_text = pd.read_csv('data/2024_winter_text.csv')
spring_2024_text = pd.read_csv('data/2024_spring_text.csv')
summer_2024_text = pd.read_csv('data/2024_summer_text.csv')
fall_2024_text = pd.read_csv('data/2024_fall_text.csv')

In [4]:
# Initial Concat
text = pd.concat([fall_2022_text, winter_2023_text, spring_2023_text, summer_2023_text, fall_2023_text, winter_2024_text, spring_2024_text, summer_2024_text, fall_2024_text], axis=0)
clean_text = text.pipe(transform_text)
clean_text

,Time,Study Materials
0,2022-02-14,"Book read (30m), DOC 2 P3 part2 (30m), Terms o..."
1,2022-02-15,"Nutrition read (15m), Physics Light cone study..."
2,2022-10-21,
3,2022-10-22,"After consecutively doing DSC 10 hw, efficienc..."
4,2022-10-23,Wolfram good to check solution | Group Coding...
...,...,...
628,2024-12-11,"RPLH eval + paper algorithm + RPLH eval, track..."
629,2024-12-12,"track-mjx meeting, RPLH paper writing, chat, c..."
630,2024-12-13,"constraint article, EM article, meet elliot, ..."
631,2024-12-14,chill


In [5]:
# pre_process = (clean_text['Study Materials']
#               .str.lower()
#               .str.replace(r'\([\d]*m\)','',regex=True)
#               .str.replace(',','')
#               .str.strip())

# corpus = ' '.join(pre_process.astype(str).to_list())
# tokens = nltk.tokenize.word_tokenize(corpus, language='english')
# len(tokens)

In [10]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
import os

# Download NLTK data files (if not already downloaded)
nltk.download('punkt')

# Initialize Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose other models as needed

# Load your DataFrame
df = pd.read_csv('your_data.csv')  # Replace with your actual data source

# Extract the text column
documents = df['text_column'].tolist()  # Replace 'text_column' with your actual column name

# Function to split text into chunks
def split_text_nltk(text, max_sentences=50):
    """
    Splits text into chunks based on a maximum number of sentences.
    """
    sentences = sent_tokenize(text)
    chunks = []
    for i in range(0, len(sentences), max_sentences):
        chunk = ' '.join(sentences[i:i + max_sentences])
        chunks.append(chunk)
    return chunks

# Split all documents into chunks
all_chunks = []
for doc in documents:
    chunks = split_text_nltk(doc)
    all_chunks.extend(chunks)

print(f"Total chunks created: {len(all_chunks)}")

# Generate embeddings for each chunk
embeddings = model.encode(all_chunks, show_progress_bar=True, convert_to_tensor=False)

# Add embeddings to a new DataFrame
embedding_df = pd.DataFrame({
    'chunk': all_chunks,
    'embedding': embeddings.tolist()
})

# Save embeddings to a file (optional)
embedding_df.to_csv('embeddings.csv', index=False)
print("Embeddings saved to embeddings.csv")

NameError: name '_C' is not defined